In [1]:
using StatisticalRethinking, Distributed, JLD
using Mamba

# Data
line = Dict{Symbol, Any}()

howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [2]:
df2 = filter(row -> row[:age] >= 18, df);
line[:x] = convert(Array{Float64,1}, df2[:weight]);
line[:y] = convert(Array{Float64,1}, df2[:height]);
line[:xmat] = convert(Array{Float64,2}, [ones(length(line[:x])) line[:x]])

# Model Specification
model = Model(
  y = Stochastic(1,
    (xmat, beta, s2) -> MvNormal(xmat * beta, sqrt(s2)),
    false
  ),
  beta = Stochastic(1, () -> MvNormal([178, 0], [sqrt(10000), sqrt(100)])),
  s2 = Stochastic(() -> Uniform(0, 50))
)

# Initial Values
inits = [
  Dict{Symbol, Any}(
    :y => line[:y],
    :beta => [rand(Normal(178, 100)), rand(Normal(0, 10))],
    :s2 => rand(Uniform(0, 50))
  )
  for i in 1:3
]

# Tuning Parameters
scale1 = [0.5, 0.25]
summary1 = identity
eps1 = 0.5

scale2 = 0.5
summary2 = x -> [mean(x); sqrt(var(x))]
eps2 = 0.1

# Define sampling scheme

scheme = [
  Mamba.NUTS([:beta]),
  Mamba.Slice([:s2], 10)
]

setsamplers!(model, scheme)

# MCMC Simulation

sim = mcmc(model, line, inits, 10000, burnin=1000, chains=3)

MCMC Simulation of 10000 Iterations x 3 Chains...

Chain 1:   0% [0:12:37 of 0:12:38 remaining]
Chain 1:  10% [0:01:03 of 0:01:10 remaining]
Chain 1:  20% [0:00:58 of 0:01:13 remaining]
Chain 1:  30% [0:00:53 of 0:01:15 remaining]
Chain 1:  40% [0:00:47 of 0:01:18 remaining]
Chain 1:  50% [0:00:40 of 0:01:19 remaining]
Chain 1:  60% [0:00:31 of 0:01:19 remaining]
Chain 1:  70% [0:00:23 of 0:01:17 remaining]
Chain 1:  80% [0:00:15 of 0:01:16 remaining]
Chain 1:  90% [0:00:08 of 0:01:16 remaining]
Chain 1: 100% [0:00:00 of 0:01:16 remaining]

Chain 2:   0% [0:00:02 of 0:00:02 remaining]
Chain 2:  10% [0:00:43 of 0:00:48 remaining]
Chain 2:  20% [0:00:46 of 0:00:57 remaining]
Chain 2:  30% [0:00:43 of 0:01:01 remaining]
Chain 2:  40% [0:00:38 of 0:01:04 remaining]
Chain 2:  50% [0:00:33 of 0:01:06 remaining]
Chain 2:  60% [0:00:27 of 0:01:06 remaining]
Chain 2:  70% [0:00:21 of 0:01:09 remaining]
Chain 2:  80% [0:00:14 of 0:01:11 remaining]
Chain 2:  90% [0:00:07 of 0:01:12 remaining]
Cha

Object of type "Mamba.ModelChains"

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

[29.291 111.11 0.971949; 31.9401 111.111 0.966764; … ; 23.7928 114.23 0.902335; 25.7125 115.853 0.864817]

[26.2135 113.438 0.917663; 26.023 116.192 0.853881; … ; 26.9004 114.421 0.883175; 25.9337 114.42 0.898069]

[27.3121 114.802 0.881749; 27.0825 115.883 0.862978; … ; 23.5384 111.898 0.941677; 25.2053 113.552 0.906353]

Show draws summary

In [3]:
describe(sim)

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

Empirical Posterior Estimates:
            Mean         SD        Naive SE       MCSE        ESS   
     s2  26.15063474 1.98833081 0.01210059597 0.0200282874 9000.0000
beta[1] 113.96980409 1.92774609 0.01173188912 0.0377786031 2603.8015
beta[2]   0.90303842 0.04245238 0.00025835696 0.0008251609 2646.8349

Quantiles:
            2.5%        25.0%       50.0%        75.0%        97.5%   
     s2  22.5516645  24.77009540  26.0489604  27.44295881  30.25827774
beta[1] 110.3251882 112.67293041 113.9333138 115.23613739 117.82433913
beta[2]   0.8186605   0.87517508   0.9036455   0.93158346   0.98418933



End of `clip_38.0m.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*